In [ ]:
!source activate NLP

In [2]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import spacy
from spacy.tokens import Doc, Span, Token
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import random
from datetime import date, timedelta
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import shutil
import glob
import webbrowser
from spacytextblob.spacytextblob import SpacyTextBlob
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Aqqes/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
# Use to download additional Spacy models
#spacy.cli.download("en")

In [ ]:
username = os.getlogin()
os.getcwd()

In [ ]:
# Use if on Home Desktop
#os.chdir("\\Users\\%s\\Documents\\GitHub\\finance\\google_news_stock_sentiment" %(username))
os.chdir("/Users/%s/Documents/GitHub/bing_news_sentiment_analysis" %(username))

In [ ]:
# Defining what to search for and what website to search for it on
search_query = 'IBM'
html_files_folder = "bing_news_html_files"

In [ ]:
# Creating empty dataframe and lists to store data
article_df = pd.DataFrame()
article_df['article_id'] = ""
article_df['headline'] = ""
article_df['time'] = ""
article_df['retrieval_date'] = ""
article_df['description'] = ""
article_df['link'] = ""
title_list = []
description_list = []
time_list = []
link_list = []
article_id_list = []

In [ ]:
#Navigating through the pages by selecting the next button
import time
driver = webdriver.Chrome(executable_path='Chrome_Driver/chromedriver')
driver.get("https://www.bing.com/news/search?q=%s"\
%(search_query))

SCROLL_PAUSE_TIME = 1

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


file = open("%s_bing_query.html"%(search_query), 'w')
file.write(driver.page_source)
file.close()
shutil.move("%s_bing_query.html"%(search_query),\
               "%s/%s_bing_query.html"%(html_files_folder,search_query))


driver.close()

In [ ]:
html_file_list = glob.glob("bing_news_html_files/*%s*.html"%(search_query))
html_file_list

In [ ]:
# Note that google news HTML references may change at any time
# In order for this cell to work you may need to update the item.find lines to find the most recent data
for file in html_file_list:
    soup = BeautifulSoup(open(file))
    for item in soup.find_all('div', attrs={'class':'news-card newsitem cardcommon b_cards2'}):
        raw_link = (item.find('a',href=True)['href'])
        link = raw_link
        title = (item.find('a',attrs={'class':"title"}).get_text())
        article_info = (item.find('div',attrs={'class':"snippet"}).get_text())
        time = (item.find('span',attrs={'tabindex':"0"}).get_text())
        description = article_info
        title_list.append(title)
        description_list.append(description)
        time_list.append(time)
        link_list.append(link)
        article_id_list.append(random.randint(10000,99999))
        print(title)

In [ ]:
# Adding data to dataframe
article_df['headline'] = title_list
article_df['time'] = time_list
article_df['description'] = description_list
article_df['link'] = link_list
article_df['article_id'] = article_id_list
article_df['retrieval_date'] = (date.today()).strftime("%m/%d/%Y")

In [ ]:
print("Total Articles Retrieved:",len(article_df))

In [ ]:
article_df.head()

In [ ]:
#Create a function to get the sentiment
sid = SentimentIntensityAnalyzer()

def get_sentiment(doc):
    sentiment_dict = sid.polarity_scores(doc.text)
    sentiment_score = sentiment_dict['compound']
    doc._.set("sentiment_score", sentiment_score)
    if sentiment_score == 0:
        doc._.set("sentiment_category", "neutral")
    elif sentiment_score > 0:
        doc._.set("sentiment_category", "positive")
    elif sentiment_score < 0:
        doc._.set("sentiment_category", "negative")
    else:
        pass
    return doc

In [3]:
nlp = spacy.load('en_core_web_sm')
Doc.set_extension("sentiment_score", default=None, force=True)
Doc.set_extension("sentiment_category", default=None, force=True)
nlp.add_pipe(get_sentiment, last=True)
print(nlp.pipe_names)

/opt/anaconda3/envs/NLP/lib/python3.7/site-packages/spacy/util.py:732: UserWarning: [W095] Model 'en_core_web_sm' (2.3.1) was trained with spaCy v2.3 and may not be 100% compatible with the current version (3.1.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


OSError: [E053] Could not read config.cfg from /opt/anaconda3/envs/NLP/lib/python3.7/site-packages/en_core_web_sm/en_core_web_sm-2.3.1/config.cfg

In [ ]:
def calculate_sentiment_score(text):
    doc = nlp(text)
    return doc._.sentiment_score

In [ ]:
def get_article_date(time_text):
    if "d" in time_text:
        days_to_subtract = int(time_text[0])
        article_date = (datetime.today() - timedelta(days=days_to_subtract)).strftime("%m/%d/%Y")
    elif 'h' in time_text:
        hours_to_subtract = int(time_text[0])
        days_to_subtract  = hours_to_subtract/24
        article_date = (datetime.today() - timedelta(days=days_to_subtract)).strftime("%m/%d/%Y")
    elif 'mon' in time_text:
        months_to_subtract = int(time_text[0])
        days_to_subtract  = months_to_subtract * 30
        article_date = (datetime.today() - timedelta(days=days_to_subtract)).strftime("%m/%d/%Y")
    else:
        article_date = None
    return article_date


In [ ]:
article_df['headline_sentiment'] = article_df['headline'].apply(calculate_sentiment_score)

In [ ]:
article_df['description_sentiment'] = article_df['description'].apply(calculate_sentiment_score)

In [ ]:
article_df['article_date'] = article_df['time'].apply(get_article_date)

In [ ]:
article_df.head()

In [ ]:
from yahoo_fin import stock_info as si

In [ ]:
# What specific entities were mentioned in the article descriptions?
# for i in article_df['description'].tolist():
#     doc = nlp(i)
#     for entity in doc.ents:
#          print(entity.text,":",entity.label_)

In [ ]:
# What do each of the entity tags in these articles represent?
# entity_tags =[]
# for i in article_df['description'].tolist():
#     doc = nlp(i)
#     for entity in doc.ents:
#         entity_tags.append(entity.label_)

# entity_tags = set(entity_tags)
# for tag in entity_tags:
#     print(tag,":",spacy.explain(tag))
      

In [ ]:
# What is the sentiment socre associated with each articles description?
# x = 0
# for description in article_df['description'].tolist():
#     doc = nlp(description)
#     print(article_df['title'][x],':', doc._.sentiment_score)
#     x+=1